## Chapter 12

### Chain of responsibility

In [1]:
class InvoiceHandler(object):
    def __init__(self):
        self.next_action = None
    
    def handle(self,print_invoice):
        self.next_action.handle(print_invoice)

class InputState(object):
    state_ny = ['NYC','NY','New York','new york']
    state_ca = ['CA', 'California', 'california']
    
class print_invoice(object):
    def __init__(self,state):
        self.state = state
        self.header = 'State specific Sales tax is applicable for the state of ' + self.state

class NYCHandler(InvoiceHandler):
    def generate_invoice(self, header, state):
        product = 'WashingMachine'
        pricebeforetax = 450 + (450 * 0.19)
        tax_rate = 0.4
        local_rate = 0.055
        tax = pricebeforetax * (tax_rate + local_rate)
        finalsellingprice = pricebeforetax + tax
        print('**************ABC Megamart*****************')
        print('***********------------------**************')
        print(header)
        print('Product: ', product)
        print('Tax: ', tax)
        print('Total Price: ', finalsellingprice)
        print('***********------------------**************') 
        
    def handle(self,print_invoice):
        if print_invoice.state in InputState.state_ny:
            self.generate_invoice(print_invoice.header, print_invoice.state)
        
        else:
            super(NYCHandler, self).handle(print_invoice)
            
class CAHandler(InvoiceHandler):
    def generate_invoice(self, header, state):
        product = 'WashingMachine'
        pricebeforetax = 480 + (480 * 0.14)
        tax_rate = 0.35
        local_rate = 0.077
        tax = pricebeforetax * (tax_rate + local_rate)
        finalsellingprice = pricebeforetax + tax
        print('**************ABC Megamart*****************')
        print('***********------------------**************')
        print(header)
        print('Product: ', product)
        print('Tax: ', tax)
        print('Total Price: ', finalsellingprice)
        print('***********------------------**************') 
        
    def handle(self,print_invoice):
        if print_invoice.state in InputState.state_ca:
            self.generate_invoice(print_invoice.header, print_invoice.state)
        
        else:
            super(CAHandler, self).handle(print_invoice)

class ExceptionHandler(InvoiceHandler):
    def handle(self,print_invoice):
        print("No branches in the state")  
   

In [2]:
def invoice_requestor(state):
    invoice = print_invoice(state)
    nychandler = NYCHandler()
    cahandler = CAHandler()

    nychandler.next_action = cahandler
    cahandler.next_action = ExceptionHandler()
    nychandler.handle(invoice)

In [3]:
invoice_requestor('CA')

**************ABC Megamart*****************
***********------------------**************
State specific Sales tax is applicable for the state of CA
Product:  WashingMachine
Tax:  233.6544
Total Price:  780.8544
***********------------------**************


In [4]:
invoice_requestor('NYC')

**************ABC Megamart*****************
***********------------------**************
State specific Sales tax is applicable for the state of NYC
Product:  WashingMachine
Tax:  243.6525
Total Price:  779.1525
***********------------------**************


In [5]:
invoice_requestor('TEXAS')

No branches in the state


### Command

In [6]:
from abc import ABC, abstractmethod

class Billing:
    sales = {'purchase_price': 450,
              'profit_margin': 0.19,
              'tax_rate': 0.4,
              'discount_rate': 0.10
              }        

    @abstractmethod
    def apply_discount(self):
        pass
    
    @abstractmethod
    def remove_discount(self):
        pass

In [7]:
class DiscountedBilling(Billing):

    def apply_discount(self):
        sales = self.sales
        pricebeforetax = sales['purchase_price'] + sales['purchase_price'] * sales['profit_margin']
        finalsellingprice = pricebeforetax + (pricebeforetax * sales['tax_rate'])
        sales['sellingPrice'] = finalsellingprice
        discountedPrice = sales['sellingPrice'] * (1 - sales['discount_rate'])
        return discountedPrice

In [8]:
class ActualBilling(Billing):

    def remove_discount(self):
        sales = self.sales
        pricebeforetax = sales['purchase_price'] + sales['purchase_price'] * sales['profit_margin']
        actualprice = pricebeforetax + (pricebeforetax * sales['tax_rate'])
        return actualprice

In [9]:
class ExecuteBilling:
    @abstractmethod
    def exec_discount(self):
        pass

    @abstractmethod
    def revoke_discount(self):
        pass

In [10]:
class ExecuteDiscountedBilling(ExecuteBilling):
    def __init__(self, instance):
        self.instance = instance
        
    def exec_discount(self):
        print('Discount applied...')
        return self.instance.apply_discount()
        
    def revoke_discount(self, revokeInstance):
        revokeInstance.reset(ExecuteActualBilling(ActualBilling()))
        return revokeInstance.runcalc()

In [11]:
class ExecuteActualBilling(ExecuteBilling):
    def __init__(self, instance):
        self.instance = instance
        
    def exec_discount(self):
        print('Discount removed...')
        return self.instance.remove_discount()
        
    def revoke_discount(self, revokeInstance):
        revokeInstance.reset(ExecuteDiscountedBilling(DiscountedBilling()))
        return revokeInstance.runcalc()

In [12]:
class RequestAction:
    def __init__(self, action):
        self.action = action

    def reset(self, action):
        print("Resetting command...")
        self.action = action

    def runcalc(self):
        return self.action.exec_discount()
    
    def revert(self):
        print("Reverting the previous action...")
        return self.action.revoke_discount(self)

In [13]:
class Tester:
    def __init__(self):
        billing = Billing()

        discount = ExecuteDiscountedBilling(DiscountedBilling())
        actual = ExecuteActualBilling(ActualBilling())
        requestor = RequestAction(discount)  

        print(requestor.runcalc())

        requestor.reset(actual)

        print(requestor.runcalc())

        print(requestor.revert())

        print(requestor.revert()) 

In [14]:
Tester()

Discount applied...
674.73
Resetting command...
Discount removed...
749.7
Reverting the previous action...
Resetting command...
Discount applied...
674.73
Reverting the previous action...
Resetting command...
Discount removed...
749.7


### Strategy pattern

In [15]:
class SuperMarket():
    
    def __init__(self,STRATEGY, items, name, scan, units, tax, itemtype = None):
        self.STRATEGY = STRATEGY
        self.items = items
        self.name = name
        self.scan = scan
        self.units = units
        self.tax = tax
        self.itemtype = itemtype
       
    def return_cart(self):
        cartItems = []
        for i in self.items:
            cartItems.append(i)
        return cartItems
    
    def goto_counter(self):
        countername = self.name
        return countername

    def scan_bar_code(self):
        codes = []
        for i in self.scan:
            codes.append(i)
        return codes
          
    def add_billing(self):
        self.codes = self.scan_bar_code()
        pricetag = []
        for i in self.units:
            pricetag.append(i)
        bill = dict(zip(self.codes, pricetag))
        return bill
        
    def add_tax(self):
        taxed = []
        for i in self.tax:
            taxed.append(i)
        return taxed
    
    def calc_bill(self):
        bill = self.add_billing()
        items = []
        cartItems = self.return_cart()
        calc_bill = []
        taxes = self.add_tax()
        for item,tax in zip(bill.items(),taxes):
            items.append(item[1])
            calc_bill.append(item[1] + item[1]*tax)
        finalbill = dict(zip(cartItems, calc_bill))
        return finalbill
        
    def print_invoice(self):
        finalbill = self.calc_bill()
        final_total = sum(finalbill.values())
        print('**************ABC Megamart*****************')
        print('***********------------------**************')
        print('Counter Name: ', self.goto_counter())
        for item,price in finalbill.items():
            print(item,": ", price)
        print('Total:',final_total)
        print('***********------------------**************')
        print('***************PAID************************')
         
    def pipeline_template(self):
        self.return_cart()
        self.goto_counter()
        self.STRATEGY.redirect_counter()
        self.scan_bar_code()
        self.add_billing()
        self.add_tax()
        self.calc_bill()
        self.print_invoice()

In [16]:
class VegeCounter():
    def redirect_counter():
        print("**************Move to Vege Counter**************")

In [17]:
class ElectronicsCounter():
    def redirect_counter():
        print("**************Move to Electronics Counter**************")

In [18]:
def run_pipeline(domain = SuperMarket):
    domain.pipeline_template()

In [19]:
run_pipeline(SuperMarket(STRATEGY = VegeCounter,
                           items = ['Onions','Tomatoes','Cabbage','Beetroot'],
                         name = ['Vegetable Counter'],
                         scan = [113323,3434332,2131243,2332783],
                         units = [10,15,12,14],
                         tax = [0.04,0.03,0.035,0.025],
                         itemtype = ['Vegetables'],
                         ))

**************Move to Vege Counter**************
**************ABC Megamart*****************
***********------------------**************
Counter Name:  ['Vegetable Counter']
Onions :  10.4
Tomatoes :  15.45
Cabbage :  12.42
Beetroot :  14.35
Total: 52.620000000000005
***********------------------**************
***************PAID************************


In [20]:
run_pipeline(SuperMarket(STRATEGY = ElectronicsCounter,
                         items = ['television','keyboard','mouse'],
                                name = ['Electronics Counter'],
                                scan = [113323,3434332,2131243],
                                units = [100,16,14],
                                tax = [0.04,0.03,0.035],
                                itemtype = ['Electronics'],
                                ))

**************Move to Electronics Counter**************
**************ABC Megamart*****************
***********------------------**************
Counter Name:  ['Electronics Counter']
television :  104.0
keyboard :  16.48
mouse :  14.49
Total: 134.97
***********------------------**************
***************PAID************************


### These are all the examples covered in this chapter.